found here : https://github.com/ImperialNLP/NLPLabs/blob/c724834960345085690802233966682bc3321723/lab06/lab06_solutions.ipynb

In [ ]:
%%bash

# Download the corpus
URL="https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/tok"

#cd data

for split in "train" "val" "test_2016_flickr"; do
    for lang in en de fr; do
        fname="${split}.lc.norm.tok.${lang}"
        if [ ! -f $fname ]; then
            echo "Downloading $fname"
            wget -q "${URL}/$fname" -O "${split/_2016_flickr/}.${lang}"
        fi
    done
done
echo 

# Print the first 10 lines with line numbers of 
# the English and French training data
cat -n train.en | head -n10
echo
cat -n train.fr | head -n10
echo
cd ..

In [ ]:
import numpy as np

In [ ]:
class Multi30K:
    """A dataset wrapper for Multi30K."""
    def __init__(self, tokenizer, src_file, trg_file):

        self.tokenizer = tokenizer
    
        self.src_sents, self.trg_sents = self.read_sentences(src_file, trg_file)

    def read_sentences(self, src_file, trg_file):
        src_sents = []
        trg_sents = []

        # Read source side
        with open(src_file) as f:
            for line in f:
                line = line.strip()
                src_sents.append(line) 
            
        # Read target side
        with open(trg_file) as f:
            for line in f:
                line = line.strip()
                trg_sents.append(line)

        assert len(src_sents) == len(trg_sents), "Files are not aligned!"
        return src_sents, trg_sents
    
    def collate_fn(self, idx):
        src_texts = [self.src_sents[i] for i in idx]
        trg_texts = [self.trg_sents[i] for i in idx]
        
        output = self.tokenizer.prepare_seq2seq_batch(src_texts=src_texts, 
                                                      tgt_texts=trg_texts, 
                                                      max_length=128, 
                                                      max_target_length=128,
                                                      return_tensors='pt',
                                                      truncation=True)
        return output
    
    def __len__(self):
        return len(self.src_sents)

    def __getitem__(self, idx):
        return idx

In [ ]:
from transformers import EncoderDecoderModel, MarianMTModel, MarianTokenizer, BartModel, BartConfig, BertConfig, BartForCausalLM,Trainer,TrainingArguments
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-de')
model

In [ ]:
def main_mt():
    
    ## QUESTION 5 ##

    mt_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')
    mt_dataset = Multi30K(mt_tokenizer, 'train.en', 'train.de')
    mt_test_dataset = Multi30K(mt_tokenizer, 'test.en', 'test.de')
    
    model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-de')

    training_args = TrainingArguments(
        output_dir='./experiment/mt',
        learning_rate = 0.00005,
        logging_steps= 5000,
        save_steps = 10000,
        num_train_epochs = 1,
        per_device_train_batch_size=2
    )
    trainer = Trainer(
        model=model,                         
        args=training_args,                 
        train_dataset=mt_dataset,                     
        data_collator=mt_dataset.collate_fn
    )

    trainer.train()

    ## when you already trained your model and want to start from a checkpoint
    #trainer.train("./experiment/mt/checkpoint-40000")

    trainer.save_model('./models/mt_marianmt/')

In [ ]:
main_mt()

In [ ]:
import sacrebleu

def evaluate_mt(model,mt_tokenizer, mt_test_dataset):

  bleu = []

  model.eval()

#   for file in tqdm(range(len(mt_test_dataset))):
  for file in range(len(mt_test_dataset)):

    src_text = mt_test_dataset.src_sents[file]
    targ_text_origin = mt_test_dataset.trg_sents[file]

    translated = model.generate(**mt_tokenizer.prepare_seq2seq_batch(src_text, return_tensors="pt"))
    translated_text = [mt_tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    bleu.append(sacrebleu.corpus_bleu(translated_text, targ_text_origin, force=True).score)

  bleu = np.asarray(bleu)

  return np.average(bleu)

In [ ]:
model = MarianMTModel.from_pretrained('./models/mt_marianmt/')

mt_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')
mt_test_dataset = Multi30K(mt_tokenizer, 'test.en', 'test.de')

bleu = evaluate_mt(model,mt_tokenizer, mt_test_dataset)

print(bleu)

In [ ]:
mt_test_dataset